<a href="https://colab.research.google.com/github/LIMDANBI/Hackathon/blob/main/KoBERT_fine_tuning(%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%87%BC%ED%95%91_%EB%A6%AC%EB%B7%B0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 환경 설정

In [1]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-qvsvugwg
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-qvsvugwg
     |████████████████████████████████| 129 kB 14.2 MB/s 
     |████████████████████████████████| 344 kB 99.9 MB/s 
     |████████████████████████████████| 54.7 MB 3.2 MB/s 
     |████████████████████████████████| 4.5 MB 84.0 MB/s 
     |████████████████████████████████| 1.2 MB 83.9 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.1 MB/s eta 0:00:43tcmalloc: large alloc 1147494400 bytes == 0x38ef0000 @  0x7f3373d53615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8

In [2]:
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
import gluonnlp as nlp
from tqdm.notebook import tqdm

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

In [3]:
device = torch.device("cuda:0")
device

device(type='cuda', index=0)

# 데이터 불러오기

In [25]:
# -*- coding: cp949 -*-
data = pd.read_csv("/content/naverReview.csv")
data

,star,review
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ
...,...,...
199995,2,장마라그런가!!! 달지않아요
199996,5,다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...
199997,5,로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요
199998,5,넘이쁘고 쎄련되보이네요~


# 0과 1로 레이블 변경

In [26]:
def changeTo01(x):
  if x<3:
    return 0
  else:
    return 1

data['star'] = data['star'].apply(changeTo01)
data

,star,review
0,1,배공빠르고 굿
1,0,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,1,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,0,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,1,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ
...,...,...
199995,0,장마라그런가!!! 달지않아요
199996,1,다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...
199997,1,로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요
199998,1,넘이쁘고 쎄련되보이네요~


In [27]:
data['star'].value_counts()

0    100037
1     99963
Name: star, dtype: int64

# Input for BERTDataset

In [28]:
data_list = []

for review, label in zip(data['review'], data['star']):
  data = []
  data.append(review)
  data.append(label)
  data_list.append(data)

len(data_list)

200000

In [29]:
data_list[:5]

[['배공빠르고 굿', 1],
 ['택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고', 0],
 ['아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 엉성하긴 하지만 편하고 가성비 최고예요.', 1],
 ['선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전화했더니 바로주신다했지만 배송도 누락되어있었네요.. 확인안하고 바로 선물했으면 큰일날뻔했네요..이렇게 배송이 오래걸렸으면 사는거 다시 생각했을거같아요 아쉽네요..',
  0],
 ['민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ', 1]]

# Train, Test Split

In [30]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data_list, test_size = 0.2, shuffle=True, random_state = 0)

In [31]:
print(len(train))
train[:5]

160000


[['두피 가려움이 심했는데 쓰고나서 가려움이 덜 하네요~머리카락도 덜 빠지고요~', 1],
 ['재구매 지난번 주문에서 만족해서, 4천원 더 비싼 특상 시켰더니 대짜 사이즈로 보내옴. 잘못 주문했나 다시 확인해봐도 특상 맞음. 지난번 시켰을때 남은것 1개와 비교 사진.',
  0],
 ['광택이 그렇게 있는것도 아니고 3번을 발라봤지만 3번다 2일 3일 지남 떨어져요 설겆이 하는데 장갑에 빠져 있어서 놀랬어요~~ 가격이 저렴한것도 아니고 별로네요',
  0],
 ['몸통사이즈는 맞는데 컵이엄청작아요 캡도붙어잇어서 c컵d컵이신분들은캡이반밖에 안가려져요 패드따로 있는걸러 살걸그랫네요 밑가슴밴드도 조금떠요 가슴줄 양옆으로 밴드벌어져요조절ㅇ안되서',
  0],
 ['배송이 늦습니다 배송상태도 실망입니다', 0]]

In [32]:
print(len(test))
test[:5]

40000


[['재구매 오랫동안 잘 썼는데 이번에 사고 아주 실망입니다 .열흘동안 배송도 안해주고 전화도 안받고 언제 보낸다 말고없고 뭐하자는건지 최악입니다. 다른곳에서는 주문하고 정상적으로 받았구요 ㅇ제 여기서 살일은 없네요',
  0],
 ['간편하고 좋아요 간단하게 이것저것 넣어다니기 짱 짱 편 합니다. 귀엽고 좋아요 ~^^ 추천합니다 만족 만족', 1],
 ['두개시켜서 찜해서 그냥먹엇는데 전..비렷어요 그냥 특히머리..머리쪽좋아해서 버터랑설탕너서버터구이해메엇는데..비리더라구요 비린거 어케잡아요? 아직남앗는데..',
  0],
 ['군밤 만들어 먹으려고 샀는데 아주 좋아요', 1],
 ['좋습니다 현존 최고 디자인 가성비 안전화입니다', 1]]

## KoBERT tokenizer, model

[CLS] : classification token (문장의 시작)

[SEP] : seperate (문장 구분)

[MASK] : MASK

[PAD] : PADDING

In [33]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [34]:
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [35]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [15]:
tok.vocab

Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")

In [16]:
vocab

Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")

# Dataset

In [36]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [i[label_idx] for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

# Parameter

In [37]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [19]:
# for i in train:
#   print(i[0])
#   print(i[1])
#   break

In [38]:
train_dataset = BERTDataset(train, 0, 1, tok, max_len, True, False)
test_dataset = BERTDataset(test, 0, 1, tok, max_len, True, False)

In [21]:
train_dataset.labels[:3]

[0, 1, 1]

In [22]:
test_dataset.sentences[:3]

[(array([   2,  517, 7668, 7668, 7819, 3135, 5628, 5199, 6060, 6999,    3,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
  array(11, dtype=int32),
  array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        dtype=int32)),
 (array([   2, 1567, 5401, 6371, 3528, 7088, 6629, 7142, 5384, 3930, 5439,
          728, 5814,  517, 6357, 6957, 6999,  517,   54,  517,   54,  517,
           54,    3,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,   

# DataLoader

In [39]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=5)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=5)

# KoBERT 모델

In [40]:
class BERTClassifier(nn.Module):
  def __init__(self, bert, hidden_size=768, num_classes=2, dr_rate=None, params=None):
    super(BERTClassifier, self).__init__()
    self.bert = bert
    self.dr_rate = dr_rate

    self.classifier = nn.Linear(hidden_size , num_classes)
    if dr_rate:
      self.dropout = nn.Dropout(p=dr_rate)

  def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i, v in enumerate(valid_length):
      attention_mask[i][:v] = 1
    return attention_mask.float()

  def forward(self, token_ids, valid_length, segment_ids):
    attention_mask = self.gen_attention_mask(token_ids, valid_length)
    _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
    if self.dr_rate:
        out = self.dropout(pooler)
    else:
        out = pooler
    return self.classifier(out)

In [41]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# optimizer and schedule

In [42]:
no_decay = ['bias', 'LayerNorm.weight']

# 최적화해야 할 parameter를 optimizer에게 알려야 함
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [43]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate) # optimizer
loss_fn = nn.CrossEntropyLoss() # loss function 

In [44]:
t_total = len(train_loader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [45]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# Calculate Accuracy

In [46]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# Train

In [47]:
for e in range(num_epochs):
    
    train_acc = 0.0
    test_acc = 0.0
    
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_loader), total=len(train_loader)):
        optimizer.zero_grad()
        
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_loader), total=len(test_loader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/2500 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.7894543409347534 train acc 0.453125
epoch 1 batch id 201 loss 0.40286409854888916 train acc 0.6479322139303483
epoch 1 batch id 401 loss 0.2667944133281708 train acc 0.7708463216957606
epoch 1 batch id 601 loss 0.11858896911144257 train acc 0.8175436772046589
epoch 1 batch id 801 loss 0.16790607571601868 train acc 0.842833177278402
epoch 1 batch id 1001 loss 0.09502232819795609 train acc 0.8572833416583416
epoch 1 batch id 1201 loss 0.13143238425254822 train acc 0.8674932348043297
epoch 1 batch id 1401 loss 0.41631507873535156 train acc 0.8747323340471093
epoch 1 batch id 1601 loss 0.2998921871185303 train acc 0.8794210649594004
epoch 1 batch id 1801 loss 0.1812306046485901 train acc 0.884491254858412
epoch 1 batch id 2001 loss 0.16679203510284424 train acc 0.8883839330334833
epoch 1 batch id 2201 loss 0.2690374255180359 train acc 0.8919738187187642
epoch 1 batch id 2401 loss 0.20890094339847565 train acc 0.8948875468554769
epoch 1 train acc 0.8961625


  0%|          | 0/625 [00:00<?, ?it/s]

epoch 1 test acc 0.929525


  0%|          | 0/2500 [00:00<?, ?it/s]

...

epoch 4 test acc 0.932825


  0%|          | 0/2500 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.09533283859491348 train acc 0.984375
epoch 5 batch id 201 loss 0.018856491893529892 train acc 0.9736473880597015
epoch 5 batch id 401 loss 0.03629876300692558 train acc 0.9731920199501247
epoch 5 batch id 601 loss 0.016179338097572327 train acc 0.9741576539101497
epoch 5 batch id 801 loss 0.017818577587604523 train acc 0.9750117041198502
epoch 5 batch id 1001 loss 0.00980259571224451 train acc 0.9751654595404595
epoch 5 batch id 1201 loss 0.013023813255131245 train acc 0.9757753955037469
epoch 5 batch id 1401 loss 0.01126093603670597 train acc 0.9761442719486081
epoch 5 batch id 1601 loss 0.13113285601139069 train acc 0.9765381011867583
epoch 5 batch id 1801 loss 0.0801263377070427 train acc 0.9767837312604108
epoch 5 batch id 2001 loss 0.08540093898773193 train acc 0.9769958770614693
epoch 5 batch id 2201 loss 0.0340057834982872 train acc 0.9771694684234439
epoch 5 batch id 2401 loss 0.15434861183166504 train acc 0.9773531861724282
epoch 5 train acc 0.977275


  0%|          | 0/625 [00:00<?, ?it/s]

epoch 5 test acc 0.93235


# 모델 저장

In [48]:
PATH = '/content/'
# torch.save(model, PATH + 'naverShoppingReview.pt')  # 전체 모델 저장
torch.save(model.state_dict(), PATH + 'naverShoppingReview_state_dict.pt')  # 모델 객체의 state_dict 저장
# torch.save({
#     'model': model.state_dict(),
#     'optimizer': optimizer.state_dict()
# }, PATH + 'all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능

# Input Test

In [49]:
def predict(predict_sentence):

    data = [predict_sentence, 0]
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_loader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_loader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("부정적")
            else:
                test_eval.append("긍정적")

        print(">> 해당 리뷰는 " + test_eval[0] + " 리뷰 입니다.")

In [54]:
while True:
    sentence = input("긍부정을 판단할 리뷰를 입력해주세요 : ")
    if sentence == "0":
      print(">> 긍부정 판단을 종료합니다!")
      break
    predict(sentence)
    print("\n")

긍부정을 판단할 리뷰를 입력해주세요 : 추천 😊
>> 해당 리뷰는 긍정적 리뷰 입니다.


긍부정을 판단할 리뷰를 입력해주세요 : 😂
>> 해당 리뷰는 부정적 리뷰 입니다.


긍부정을 판단할 리뷰를 입력해주세요 : 😭😭 다시는 안 살 듯
>> 해당 리뷰는 부정적 리뷰 입니다.


긍부정을 판단할 리뷰를 입력해주세요 : 실망이에요 ㅜ
>> 해당 리뷰는 부정적 리뷰 입니다.


긍부정을 판단할 리뷰를 입력해주세요 : 실밥이 바로 다 터져버렸어요 ,,
>> 해당 리뷰는 부정적 리뷰 입니다.


긍부정을 판단할 리뷰를 입력해주세요 : 옷 재질이 너무,, 안 좋아요
>> 해당 리뷰는 부정적 리뷰 입니다.


긍부정을 판단할 리뷰를 입력해주세요 : 옷 재질이 너무  고급 스러워요!
>> 해당 리뷰는 긍정적 리뷰 입니다.


긍부정을 판단할 리뷰를 입력해주세요 : 대 만족입니당
>> 해당 리뷰는 긍정적 리뷰 입니다.


긍부정을 판단할 리뷰를 입력해주세요 : 0
>> 긍부정 판단을 종료합니다!
